In [64]:
import pandas as pd
import numpy as np
import yfinance as yf
import time
import telegram
# from tabulate import tabulate
import matplotlib.pyplot as plt
from datetime import date
from dateutil.relativedelta import relativedelta
import asyncio
#import talib as TA

In [65]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import requests 
from bs4 import BeautifulSoup

In [66]:
# Class declaration section
class Stock:
  def __init__(self, stock_code, ticker_no, close_price, close_price_ld, close_price_l5d):
    self.stock_code,  self.ticker_no = stock_code, ticker_no
    self.close_price, self.close_price_ld = to_float(close_price), to_float(close_price_ld)
    self.close_price_l5d = to_float(close_price_l5d)
    self.close_price_diff = self.close_price - self.close_price_ld
    self.daily_return = self.calc_return(self.close_price, self.close_price_ld)
    self.five_days_return = self.calc_return(self.close_price, self.close_price_l5d)
    
  def calc_return(self, close_price, close_price_ld):
    return (to_float(100*(close_price-close_price_ld)/close_price_ld))

In [67]:
# function declaration section
def to_ticker(stock_code):
    stock_code = '0000' + stock_code
    ticker_no = stock_code[-4:] + '.HK'
    return(ticker_no)

def to_float(s):
    return(round(float(s), 4 ))

def get_stock_data(stock_code, ticker_no, init_date):
    df_stock = yf.download(ticker_no, init_date, progress=False)
    close_price = df_stock.iloc[-1]["Close"]
    close_price_ld = df_stock.iloc[-2]["Close"]
    close_price_l5d = df_stock.iloc[-5]["Close"]
    stock = Stock(stock_code, ticker_no, close_price, close_price_ld, close_price_l5d)
    return(stock)

In [72]:
# init
today=date.today()
today_diff=today - relativedelta(months=1)
today_before=today_diff.isoformat()
today=today.isoformat()

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


In [69]:
# import excel file
df_stock_list_xlsx=pd.read_excel("C:/temp/My_Stock_monitoring_list_1.xlsx")
df_stock_list_xlsx = df_stock_list_xlsx.astype(str)  # convert all columns to string type
ticker_list = [to_ticker(stock_code) for stock_code in list(df_stock_list_xlsx['Stock Code'])]

df_stock_list=df_stock_list_xlsx.copy()
df_stock_list['Ticker No']=ticker_list
df_stock_list

,Stock Code,Ticker No
0,939,0939.HK
1,941,0941.HK
2,1398,1398.HK
3,2600,2600.HK
4,2628,2628.HK


In [70]:
stocks = []
for index, row in df_stock_list.iterrows():  
    stock_code = row["Stock Code"]
    ticker_no = row["Ticker No"]
    print("Processing stock: " + ticker_no + "....")
    stock = get_stock_data(stock_code, ticker_no, today_before)
    stocks.append(stock)
print("Processing Ended")
    
df_stock_fin_list=pd.DataFrame([stock.__dict__ for stock in stocks])
df_stock_fin_list.columns = ['Stock Code', 'Ticker No', 'Latest Price', 'Last Day Price', 'Last 5 Day Price', 'Price Difference', 'Daily Return %', '5 Day Return %' ]
df_stock_fin_list

Processing stock: 0939.HK....
Processing stock: 0941.HK....
Processing stock: 1398.HK....
Processing stock: 2600.HK....
Processing stock: 2628.HK....
Processing Ended


,Stock Code,Ticker No,Latest Price,Last Day Price,Last 5 Day Price,Price Difference,Daily Return %,5 Day Return %
0,939,0939.HK,4.35,4.49,4.42,-0.14,-3.1180,-1.5837
1,941,0941.HK,64.05,64.70,65.20,-0.65,-1.0046,-1.7638
2,1398,1398.HK,3.67,3.78,3.70,-0.11,-2.9101,-0.8108
3,2600,2600.HK,3.79,3.84,3.70,-0.05,-1.3021,2.4324
4,2628,2628.HK,13.36,13.52,12.68,-0.16,-1.1834,5.3628


In [71]:
list(df_stock_fin_list['Ticker No'])

['0939.HK', '0941.HK', '1398.HK', '2600.HK', '2628.HK']

In [76]:
D_tickers = {}
L_tickers_no = list(df_stock_fin_list['Stock Code'])
L_tickers = list(df_stock_fin_list['Ticker No'])
L_tickers_curr,L_tickers_updown,L_tickers_updown_per,L_tickers_name = [],[],[],[]
for aticker in L_tickers:
    url = f'https://hk.finance.yahoo.com/quote/{aticker}?p={aticker}&.tsrc=fin-srch'
    driver.get(url)
    #time.sleep(1)
    html=driver.page_source
    text_non= html.replace("\n","")
    bsObj = BeautifulSoup(text_non, "html.parser")    
    priceDiv=bsObj.find_all("div", { "class" : "D(ib) Mend(20px)"} )
    for idx, c in enumerate(priceDiv[0].children):
        if idx == 0:
            L_tickers_curr.append(c.text) 
        if idx == 1:
            L_tickers_updown.append(c.text)
        if idx == 3:
            L_tickers_updown_per.append(c.text)
    stockName=bsObj.find("h1", { "class" : "D(ib) Fz(18px)"} )    
    L_tickers_name.append(stockName.text)
D_tickers['Stock Code']=L_tickers_no
D_tickers['Stock Name']=L_tickers_name
D_tickers['Curr Price']=L_tickers_curr
D_tickers['Up Down']=L_tickers_updown
D_tickers['Up Down %']=L_tickers_updown_per
df_realtime=pd.DataFrame(D_tickers)
df_realtime

,Stock Code,Stock Name,Curr Price,Up Down,Up Down %
0,939,建設銀行 (0939.HK),4.350,-0.140,(-3.12%)
1,941,中國移動 (0941.HK),64.050,-0.650,(-1.00%)
2,1398,工商銀行 (1398.HK),3.670,-0.110,(-2.91%)
3,2600,中國鋁業 (2600.HK),3.790,-0.050,(-1.30%)
4,2628,中國人壽 (2628.HK),13.360,-0.160,(-1.18%)


In [77]:
df_result = pd.merge(df_realtime, df_stock_fin_list, on='Stock Code')
df_result

,Stock Code,Stock Name,Curr Price,Up Down,Up Down %,Ticker No,Latest Price,Last Day Price,Last 5 Day Price,Price Difference,Daily Return %,5 Day Return %
0,939,建設銀行 (0939.HK),4.350,-0.140,(-3.12%),0939.HK,4.35,4.49,4.42,-0.14,-3.1180,-1.5837
1,941,中國移動 (0941.HK),64.050,-0.650,(-1.00%),0941.HK,64.05,64.70,65.20,-0.65,-1.0046,-1.7638
2,1398,工商銀行 (1398.HK),3.670,-0.110,(-2.91%),1398.HK,3.67,3.78,3.70,-0.11,-2.9101,-0.8108
3,2600,中國鋁業 (2600.HK),3.790,-0.050,(-1.30%),2600.HK,3.79,3.84,3.70,-0.05,-1.3021,2.4324
4,2628,中國人壽 (2628.HK),13.360,-0.160,(-1.18%),2628.HK,13.36,13.52,12.68,-0.16,-1.1834,5.3628
